# Capstone-Project Week 3 - Ex. 3 (Scroll down!)

### Import libraries

In [2]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

### Code from exercise 1 and 2

In [3]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(URL)

soup = BeautifulSoup(page, "html.parser")
right_table=soup.find('table', class_='wikitable sortable')

pos_code = []
borough = []
hood = []

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        if (cells[1].find(text=True).split("\n")[0] != 'Not assigned'):
            pos_code.append(cells[0].find(text=True).split("\n")[0])
            borough.append(cells[1].find(text=True).split("\n")[0])
            if(cells[2].find(text=True).split("\n")[0] == 'Not assigned'):
                hood.append(cells[1].find(text=True).split("\n")[0])
            else:
                hood.append(cells[2].find(text=True).split("\n")[0])
                
df_toronto = pd.DataFrame(pos_code,columns=['PostalCode'])
df_toronto['Borough']=borough
df_toronto['Neighborhood']=hood

df_coor = pd.read_csv('Geospatial_Coordinates.csv')

lat = []
long = []

for Code in df_toronto['PostalCode']:
    
    lat.append(df_coor.loc[df_coor['Postal Code'] == Code].iat[0,1])
    long.append(df_coor.loc[df_coor['Postal Code'] == Code].iat[0,2])
    
df_toronto['Latitude']=lat
df_toronto['Longitude']=long
df_toronto.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### Foursquare ID

In [4]:
CLIENT_ID = 'IT3MIH0AQK5B1TYT3DYZSB5XFJWYPYH3OPJH3N4F5A5Z4GRS'
CLIENT_SECRET = 'XN14APBJ235U5SJ0XXUG01ZQKL2NIUIDOFYWWZO2ZIB1VJZ0'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: IT3MIH0AQK5B1TYT3DYZSB5XFJWYPYH3OPJH3N4F5A5Z4GRS
CLIENT_SECRET:XN14APBJ235U5SJ0XXUG01ZQKL2NIUIDOFYWWZO2ZIB1VJZ0


### Function that loops through the Foursquare Data of Toronto

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
  
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)

        results = requests.get(url).json()["response"]['groups'][0]['items']

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Save the venues in dataframe

In [6]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

In [7]:
print(toronto_venues.shape)
toronto_venues.head()

(2146, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


### Onehotencoding

In [8]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

### Function that sortes the data

In [9]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### New Dataframe with the top 5 venues

In [10]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

### Group the data with k_means (k = 4)

In [17]:
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Create new dataframe with the coordinates and top 5 veneus of every neighborhood

In [18]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


toronto_merged = toronto_merged.dropna()

### Picture the neighborhoods on a card with color = cluster

In [19]:
map_clusters = folium.Map(location=[43.7, -79.4], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Look at the top 5 venues of the 4 clusters

The results show that Torontos neighborhoods are quite similar and can be put into two main cluster. Cluster 0 is the biggest cluster and the top venues show, that it is a mix from shops and food stores. The other big cluster is dominated by parks and other outdoor activities and places. Additionally, two neighborhoods with a baseball field are in a third cluster. 

In [20]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,North York,0.0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,French Restaurant
2,Downtown Toronto,0.0,Coffee Shop,Park,Pub,Bakery,Theater
3,North York,0.0,Furniture / Home Store,Clothing Store,Accessories Store,Miscellaneous Shop,Vietnamese Restaurant
4,Downtown Toronto,0.0,Coffee Shop,Diner,Yoga Studio,College Cafeteria,Beer Bar
5,Etobicoke,0.0,Pizza Place,Women's Store,Discount Store,Dance Studio,Deli / Bodega
...,...,...,...,...,...,...,...
96,Downtown Toronto,0.0,Coffee Shop,Café,Italian Restaurant,Park,Pizza Place
97,Downtown Toronto,0.0,Coffee Shop,Café,Hotel,Japanese Restaurant,Gym
99,Downtown Toronto,0.0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant
100,East Toronto,0.0,Light Rail Station,Garden Center,Brewery,Farmers Market,Fast Food Restaurant


In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
57,North York,1.0,Baseball Field,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant
101,Etobicoke,1.0,Pool,Baseball Field,Deli / Bodega,Department Store,Dessert Shop


In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,2.0,Park,Food & Drink Shop,Pool,Donut Shop,Doner Restaurant
21,York,2.0,Park,Pool,Women's Store,Gluten-free Restaurant,Dumpling Restaurant
35,East York,2.0,Park,Intersection,Convenience Store,Discount Store,Department Store
61,Central Toronto,2.0,Park,Bus Line,Swim School,Dumpling Restaurant,Drugstore
64,York,2.0,Park,Women's Store,Discount Store,Dance Studio,Deli / Bodega
66,North York,2.0,Park,Convenience Store,Women's Store,Discount Store,Deli / Bodega
83,Central Toronto,2.0,Park,Summer Camp,Diner,Dance Studio,Deli / Bodega
85,Scarborough,2.0,Park,Playground,Discount Store,Dance Studio,Deli / Bodega
91,Downtown Toronto,2.0,Park,Playground,Trail,Discount Store,Dance Studio
98,Etobicoke,2.0,Park,River,College Cafeteria,College Gym,Dumpling Restaurant
